<a href="https://colab.research.google.com/github/kunakl07/AL_for_Orca_detection/blob/master/SRKW_Preprocess_with_large_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%bash
apt-get install awscli

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  docutils-common python3-botocore python3-certifi python3-chardet
  python3-colorama python3-dateutil python3-docutils python3-idna
  python3-jmespath python3-olefile python3-pil python3-pkg-resources
  python3-pyasn1 python3-pygments python3-requests python3-roman python3-rsa
  python3-s3transfer python3-six python3-urllib3 python3-yaml sgml-base
  xml-core
Suggested packages:
  docutils-doc fonts-linuxlibertine | ttf-linux-libertine texlive-lang-french
  texlive-latex-base texlive-latex-recommended python-pil-doc python3-pil-dbg
  python3-setuptools ttf-bitstream-vera python3-cryptography python3-openssl
  python3-socks sgml-base-doc debhelper
The following NEW packages will be installed:
  awscli docutils-common python3-botocore python3-certifi python3-chardet
  python3-colorama python3-dateutil python3-docutils python3-idna
  python3-jmespath pytho

In [0]:
%%bash

aws --no-sign-request s3 cp s3://acoustic-sandbox/labeled-data/detection/train/OrcasoundLab07052019_PodCastRound2.tar.gz /content

download: s3://acoustic-sandbox/labeled-data/detection/train/OrcasoundLab07052019_PodCastRound2.tar.gz to ./OrcasoundLab07052019_PodCastRound2.tar.gz


In [0]:
%%bash
tar -xzvf/content/OrcasoundLab07052019_PodCastRound2.tar.gz


Round2_OS_07_05/
Round2_OS_07_05/dev.tsv
Round2_OS_07_05/invstd64.txt
Round2_OS_07_05/mean64.txt
Round2_OS_07_05/README.md
Round2_OS_07_05/train.tsv
Round2_OS_07_05/wav/
Round2_OS_07_05/wav/1562337136_0004.wav
Round2_OS_07_05/wav/1562337136_0005.wav
Round2_OS_07_05/wav/1562337136_0006.wav
Round2_OS_07_05/wav/1562337136_000a.wav
Round2_OS_07_05/wav/1562337136_000b.wav
Round2_OS_07_05/wav/1562337136_000c.wav
Round2_OS_07_05/wav/1562337136_000d.wav
Round2_OS_07_05/wav/1562337136_000e.wav
Round2_OS_07_05/wav/1562337136_0010.wav
Round2_OS_07_05/wav/1562337136_0012.wav
Round2_OS_07_05/wav/1562337136_0013.wav
Round2_OS_07_05/wav/1562337136_0017.wav
Round2_OS_07_05/wav/1562337136_0018.wav
Round2_OS_07_05/wav/1562337136_0019.wav
Round2_OS_07_05/wav/1562337136_001a.wav
Round2_OS_07_05/wav/1562337136_001b.wav
Round2_OS_07_05/wav/1562337136_001c.wav
Round2_OS_07_05/wav/1562340736_0001.wav
Round2_OS_07_05/wav/1562340736_0004.wav
Round2_OS_07_05/wav/1562340736_0005.wav
Round2_OS_07_05/wav/1562340736

In [0]:
pip install ketos==2.0.0b4


In [0]:
!pip install pysoundfile

In [0]:
import pandas as pd
from ketos.data_handling import selection_table as sl
import ketos.data_handling.database_interface as dbi
from ketos.data_handling.parsing import load_audio_representation
from ketos.audio.spectrogram import MagSpectrogram
from ketos.data_handling.parsing import load_audio_representation

In [0]:
annot_train = pd.read_csv('/content/Round2_OS_07_05/podcast2.tsv', sep='\t')


In [0]:
annot_train

,wav_filename,start_time_s,duration_s,location,date,data_source,data_source_id,label
0,1562337136_0004.wav,49.765625,2.450000,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs
1,1562337136_0004.wav,41.046007,1.658854,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs
2,1562337136_0004.wav,37.345486,1.743924,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs
3,1562337136_0004.wav,42.917535,2.594618,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs
4,1562337136_0004.wav,45.980035,2.041667,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562337136,SRKWs
...,...,...,...,...,...,...,...,...
352,1562344334_000e.wav,29.699870,2.105469,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562344334,SRKWs
353,1562344334_000e.wav,43.895833,2.488281,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562344334,SRKWs
354,1562344334_000e.wav,54.295573,1.977865,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562344334,SRKWs
355,1562344334_000f.wav,7.440918,2.105469,orcasound_lab,2019-07-05,Orcasound_PodCast_Round2,1562344334,SRKWs


In [0]:
sl.is_standardized(annot_train)


 Your table is not in the Ketos format.

            It should have two levels of indices: filename and annot_id.
            It should also contain at least the 'label' column.
            If your annotations have time information, these should appear in the 'start' and 'end' columns

            extra columns are allowed.

            Here is a minimum example:

                                 label
            filename  annot_id                    
            file1.wav 0          2
                      1          1
                      2          2
            file2.wav 0          2
                      1          2
                      2          1


            And here is a table with time information and a few extra columns ('min_freq', 'max_freq' and 'file_time_stamp')

                                 start   end  label  min_freq  max_freq  file_time_stamp
            filename  annot_id                    
            file1.wav 0           7.0   8.1      2    180.6     2

False

In [0]:
#map_to_ketos_annot_std ={'wav_filename': 'filename'} 
#std_annot_train = sl.standardize(table=annot_train, signal_labels=["start_time_s"], mapper=map_to_ketos_annot_std, trim_table=True)
#std_annot_test = sl.standardize(table=annot_test, signal_labels=["upcall"], mapper=map_to_ketos_annot_std, trim_table=True)
annot_std, label_dict = sl.standardize(annot_train, mapper={'wav_filename':'filename', 'start_time':'start_time_s', 'duration':'duration_s','label':'label'}, return_label_dict=True)


In [0]:
annot_std

start_time_s  duration_s  ... data_source_id label
filename            annot_id                            ...                     
1562337136_0004.wav 0            49.765625    2.450000  ...     1562337136     1
                    1            41.046007    1.658854  ...     1562337136     1
                    2            37.345486    1.743924  ...     1562337136     1
                    3            42.917535    2.594618  ...     1562337136     1
                    4            45.980035    2.041667  ...     1562337136     1
...                                    ...         ...  ...            ...   ...
1562344334_000e.wav 4            29.699870    2.105469  ...     1562344334     1
                    5            43.895833    2.488281  ...     1562344334     1
                    6            54.295573    1.977865  ...     1562344334     1
1562344334_000f.wav 0             7.440918    2.105469  ...     1562344334     1
1562344334_001c.wav 0            31.534653    2.165842  ...     1562344334     1

[357 rows x 7 columns]

In [0]:
spec_cfg = load_audio_representation('/content/Round2_OS_07_05/spec_config.json', name="spectrogram")


In [0]:
label_dict

{'SRKWs': 1}

In [0]:
spec_cfg

{'freq_max': 500,
 'freq_min': 0,
 'rate': 1000,
 'step': 0.032,
 'type': 'MagSpectrogram',
 'window': 0.256,
 'window_func': 'hamming'}

In [0]:
dbi.create_database(output_file='database.h5', data_dir='/content/Round2_OS_07_05/wav',
                               dataset_name='wav',selections=annot_std,
                               audio_repres=spec_cfg)
                              

100%|██████████| 357/357 [02:24<00:00,  2.46it/s]

357 items saved to database.h5


In [0]:
db = dbi.open_file("database.h5", 'r')


In [0]:
db

File(filename=database.h5, title='', mode='r', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/wav (Group) ''
/wav/data (Table(357,), fletcher32, shuffle, zlib(1)) ''
  description := {
  "data": Float32Col(shape=(1913, 129), dflt=0.0, pos=0),
  "filename": StringCol(itemsize=100, shape=(), dflt=b'', pos=1),
  "id": UInt32Col(shape=(), dflt=0, pos=2),
  "label": UInt8Col(shape=(), dflt=0, pos=3),
  "offset": Float64Col(shape=(), dflt=0.0, pos=4)}
  byteorder := 'little'
  chunkshape := (1,)

In [0]:
db.close()